In [ ]:
sentence_transformer = 'sentence-transformers/multi-qa-distilbert-cos-v1'
model_name_hugging_face = "Intel/dynamic_tinybert"
tokenizer_name_hugging_face = "Intel/dynamic_tinybert"

In [4]:
# Allowed to make changes.
import collections
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval

!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, QuestionAnsweringPipeline
from transformers import AlbertTokenizer, AlbertForQuestionAnswering, AlbertConfig, PreTrainedTokenizer
from tqdm.auto import tqdm
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e9fef8658a7a81bc633d0864248fd599483e6e753dfbde002fd389e06fd22cac
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [20]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [9]:
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.
device = "cuda" if torch.cuda.is_available() else "cpu"
input_paragraph_file = "/kaggle/input/devrev/input_paragraph.csv"#"input_paragraph.csv"
theme_interval_file = "/kaggle/input/devrev/theme_interval.csv"#"theme_interval.csv"
input_question_file = "/kaggle/input/devrev/input_question.csv"#"input_question.csv"
ground_truth_file = "/kaggle/input/devrev/ground_truth.csv"#"ground_truth.csv"
output_file = "output.csv"#"output.csv"
"""dict_theme_ind[<THEME>]['<start|end>']"""
df_paras = pd.read_csv(input_paragraph_file, index_col=0)
para_startInd = json.loads(pd.read_csv(theme_interval_file).to_json(orient="records"))


theme_para_dict = {}
dict_theme_ind = {}
dict_paras = df_paras.to_dict()
for i in range(1,len(dict_paras['theme'].keys())+1):
  if dict_paras['theme'][i] in theme_para_dict:
    theme_para_dict[dict_paras['theme'][i]].append(dict_paras['paragraph'][i])
  else:
    theme_para_dict[dict_paras['theme'][i]] = [(dict_paras['paragraph'][i])]
    dict_theme_ind[dict_paras['theme'][i]] = {"start": i}
    
retriver_model = SentenceTransformer(sentence_transformer)
passage_enc_dict = {}
for theme in tqdm(theme_para_dict):
  passage_enc_dict[theme] = retriver_model.encode(theme_para_dict[theme], show_progress_bar=False)
  
# Dummy code.
# paragraph_ds = []
# global_model = {}


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

  0%|          | 0/361 [00:00<?, ?it/s]

In [10]:
# Allowed to make changes.
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_hugging_face)
model = AutoModelForQuestionAnswering.from_pretrained(model_name_hugging_face)
model = model.to(device)
def get_theme_model(theme):
  # Here, you can load and return a model which is fine tuned for a theme.
  # In case, you only have a global model, you can return that.
  return tokenizer, model

Downloading:   0%|          | 0.00/351 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/743 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Some weights of the model checkpoint at Intel/dynamic_tinybert were not used when initializing BertForQuestionAnswering: ['fit_dense.weight', 'fit_dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# Allowed to make changes.


def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]["theme"]

  for question in questions:
    #-------------------------------------   
    # add your prediction methodology here.
    #-------------------------------------
    ans = {}
    ans["question_id"] = question["id"]

    ########### predict paragraph id ########

    id = (passage_enc_dict[theme]@retriver_model.encode(question['question'], show_progress_bar=False).reshape((-1,1))).flatten().argmax()
    para_id = dict_theme_ind[theme]["start"] + id
    ans["paragraph_id"] = para_id
    ######### predict answer ###########
    text = theme_para_dict[theme][id]
    
    inputs = tokenizer(question["question"], text, return_tensors="pt")
    if((inputs['input_ids'].size()[1])>500):
      ans["paragraph_id"] = -1
      ans["answers"] = " "
      continue
    inputss = inputs.to(device)

    
    with torch.no_grad():
        outputs = model(**inputss)
    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    ans["answers"] = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)
    if(ans["answers"] == ''):
      ans["paragraph_id"] = -1
      ans["answers"] = " "
    # print(question['question'])
    # print(ans)
    pred_out.append(ans)

In [13]:
# NOT allowed to make changes. 

# All theme prediction.
questions = json.loads(pd.read_csv(input_question_file).to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv(theme_interval_file).to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in tqdm(theme_intervals):
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv(output_file, index=False)

  0%|          | 0/361 [00:00<?, ?it/s]

In [14]:
# NOT allowed to make changes. 

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
        try:
            return text.lower()
        except:
            return " "
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
        try:
            return text.lower()
        except:
            return " "
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

In [15]:
# NOT allowed to make changes. 

# Evaluation methodology.
metrics = {}
pred = pd.read_csv(output_file)
truth = pd.read_csv(ground_truth_file)

truth.paragraph_id = [ [x] for x in truth['paragraph_id'].to_list()]
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv(input_question_file)
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]
  
  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  if predicted_paragraph in truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1


In [16]:
theme_weights = {}
for theme_interval in theme_intervals:
  theme_weights[theme_interval['theme']] = (- int(theme_interval["start"]) + int(theme_interval["end"]))/len(questions)


In [19]:
# NOT allowed to make changes. 

# Final score.
inf_time_threshold = 1000.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0

temp_para_sc = 0.0
temp_qa_sc = 0.0

# Weight would stay hidden from teams.
# theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}
for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"] 
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_para_score += theme_weights[theme] * inf_time_score * para_score
  temp_para_sc += para_score * theme_weights[theme]
  temp_qa_sc += qa_score * theme_weights[theme]
print (final_para_score)
print (final_qa_score)
print(inf_time_score)
print(temp_para_sc)
print(temp_qa_sc)

0.7161802643246218
0.44148620466489075
1.0
0.7161802643246218
0.44148620466489075


In [21]:
get_n_params(model)

66365954